In [1]:
import sounddevice as sd
import numpy as np
import pandas as pd
import datetime as dt
import os

place holders and global variables

In [2]:
SOUND_AMPLITUDE = 0
AUDIO_CHEAT = 0

sound variables<br>
SUS means next sound packet is worth analyzing

In [3]:
CALLBACKS_PER_SECOND = 38               # callbacks per sec(system dependent)
SUS_FINDING_FREQUENCY = 2               # calculates SUS *n* times every sec
SOUND_AMPLITUDE_THRESHOLD = 20          # amplitude considered for SUS calc 

packing *n* frames to calculate SUS

In [4]:
FRAMES_COUNT = int(CALLBACKS_PER_SECOND/SUS_FINDING_FREQUENCY)
AMPLITUDE_LIST = list([0]*FRAMES_COUNT)
SUS_COUNT = 0
count = 0

In [5]:
# dataframe for logging AUDIO_CHEAT
df = pd.DataFrame(columns=['Filename', 'Timestamp', 'AUDIO_CHEAT'])

In [6]:
def sound(filename):
    global AUDIO_CHEAT
    with sd.InputStream(callback=lambda indata, outdata, frames, time, status: print_sound(indata, outdata, frames, time, status)):
        sd.sleep(-1)

In [7]:
def print_sound(indata, outdata, frames, time, status):
    avg_amp = 0
    global SOUND_AMPLITUDE, SUS_COUNT, count, SOUND_AMPLITUDE_THRESHOLD, AUDIO_CHEAT
    vnorm = int(np.linalg.norm(indata)*10)
    AMPLITUDE_LIST.append(vnorm)
    count += 1
    AMPLITUDE_LIST.pop(0)
    if count == FRAMES_COUNT:
        avg_amp = sum(AMPLITUDE_LIST)/FRAMES_COUNT
        SOUND_AMPLITUDE = avg_amp
        if SUS_COUNT >= 2:
            #print("!!!!!!!!!!!! FBI OPEN UP !!!!!!!!!!!!")
            AUDIO_CHEAT = 1
            SUS_COUNT = 0
        if avg_amp > SOUND_AMPLITUDE_THRESHOLD:
            SUS_COUNT += 1
            #print("Sus...", SUS_COUNT)
        else:
            SUS_COUNT = 0
            AUDIO_CHEAT = 0
        count = 0

In [8]:
def sound_analysis():
    global AMPLITUDE_LIST, FRAMES_COUNT, SOUND_AMPLITUDE
    while True:
        AMPLITUDE_LIST.append(SOUND_AMPLITUDE)
        AMPLITUDE_LIST.pop(0)
        avg_amp = sum(AMPLITUDE_LIST)/FRAMES_COUNT
        if avg_amp > 10:
            print("Sus...")

In [ ]:
if __name__ == "__main__":
    # loop through all audio files in the dataset
    #for i in range(1, 25):  # assuming the dataset has 24 subjects
        filename = f'G:/proctoring/AudioData/Yousef.wav'  
        sound(filename)

Exception ignored from cffi callback <function _StreamBase.__init__.<locals>.callback_ptr at 0x0000022396EB8EA0>:
Traceback (most recent call last):
  File "C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sounddevice.py", line 846, in callback_ptr
    return _wrap_callback(callback, data, frames, time, status)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sounddevice.py", line 2687, in _wrap_callback
    callback(*args)
TypeError: sound.<locals>.<lambda>() missing 1 required positional argument: 'status'


In [ ]:
# log AUDIO_CHEAT to dataframe
df = pd.concat([df, pd.DataFrame({'Filename': os.path.basename(filename), 'Timestamp': [dt.datetime.now()], 'AUDIO_CHEAT': [AUDIO_CHEAT]})], ignore_index=True)